#### Importing Dependencies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import numpy as np
from shutil import copyfile
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.plotting import view_img, glass_brain, plot_anat, plot_epi
import glob
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import metrics


C:\Users\Tanya Joon\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Importing Preprossed Data CSV file

In [2]:
data=pd.read_csv('Data.csv')
data

,filename,target
0,3123_functional.nii,PD
1,3124_functional.nii,PD
2,3125_functional.nii,PD
3,3132_functional.nii,PD
4,3201_1_functional.nii,control
...,...,...
119,4116_3_functional.nii,control
120,4116_4_functional.nii,control
121,4116_5_functional.nii,control
122,4117_functional.nii,PD


#### importing data

In [5]:
ff = glob.glob('all_data/*')
len(ff)

124

#### Initializing an array to store the images

In [6]:
images = []
print(images)

[]


#### Loading images
#### Shape Correction (taking middle 30 slides for each scan)
#### appending the images into array

In [7]:
for f in range(len(ff)):
    a = nib.load(ff[f])
    print(ff[f])
    l1,l2,l3=a.shape
    #print(l1,l2,l3)
    a = a.get_data()
    #print(a)
    
    z1= int(l3/2) -15
    z2= int(l3/2) +15
    a = a[:,:,z1:z2]
    #print(a)
    #for i in range (len(ff)):
    images.append((a[:,:,:]))
    #print (a.shape)
#print(images)

all_data\3123_functional.nii
all_data\3124_functional.nii
all_data\3125_functional.nii
all_data\3132_functional.nii
all_data\3201_1_functional.nii
all_data\3201_2_functional.nii
all_data\3201_3_functional.nii
all_data\3201_4_functional.nii
all_data\3220_functional.nii
all_data\3223_functional.nii
all_data\3225_functional.nii
all_data\3228_functional.nii
all_data\3230_functional.nii
all_data\3234_functional.nii
all_data\3235_1_functional.nii
all_data\3235_2_functional.nii
all_data\3237_1_functional.nii
all_data\3237_2_functional.nii
all_data\3237_3_functional.nii
all_data\3237_4_functional.nii
all_data\3409_functional.nii
all_data\3418_functional.nii
all_data\3422_functional.nii
all_data\3423_functional.nii
all_data\3431_functional.nii
all_data\3433_functional.nii
all_data\3435_functional.nii
all_data\3436_functional.nii
all_data\3443_functional.nii
all_data\3444_functional.nii
all_data\3445_functional.nii
all_data\3448_functional.nii
all_data\3453_1_functional.nii
all_data\3453_2_funct

#### Checking Dimension of Image Stack

In [8]:
imagess=np.asarray(images)
imagess.shape

(124, 128, 128, 30)

#### Min Max Normalization

In [9]:
m = np.max(imagess)
mi = np.min(imagess)

In [10]:
print(m, mi)

32767.0 -607.0


In [11]:
images = (images - mi) / (m - mi)

In [12]:
np.min(images), np.max(images)

(0.0, 1.0)

In [13]:
imagess

array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.

#### Reading Output Labels

In [14]:
data=pd.read_csv('Datacnn.csv')
y=data.target
print(y)

0      1
1      1
2      1
3      1
4      0
      ..
119    0
120    0
121    0
122    1
123    1
Name: target, Length: 124, dtype: int64


#### Changing The 4D image Stack to a 2D Stack to feed into Random Forest Classifier

In [15]:
nsamples, nx, ny, nz = imagess.shape
train_dataset = imagess.reshape((nsamples,nx*ny*nz))

In [22]:
train_dataset

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#### Training SVM Using K=5 fold 

In [43]:
kf = KFold(n_splits=5, random_state=None, shuffle=True) # Define the split - into 2 folds 
kf.get_n_splits(train_dataset) # returns the number of splitting iterations in the cross-validator
print(kf) 

svclassifier = SVC(kernel='linear')
scores=[]
for train_index, test_index in kf.split(train_dataset):
    print('Train:', train_index, 'TEST:', test_index)
    x_train, x_test = train_dataset[train_index], train_dataset[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = svclassifier.fit(x_train, y_train)
    y_pred = svclassifier.predict(x_test)
    scores.append(model.score(x_test,y_test))
    #scores.append(accuracy_score(y_test, y_pred))
    print('Scores from each Iteration: ', scores)
print('Scores from each Iteration: ', scores)
print('Average K-Fold Score :' , np.mean(scores))

KFold(n_splits=5, random_state=None, shuffle=True)
Train: [  0   1   2   3   5   7   9  10  12  13  14  15  17  18  20  21  22  23
  24  26  28  29  30  31  32  33  34  36  37  38  39  40  41  42  43  44
  45  46  47  49  50  51  52  53  54  55  56  58  59  60  61  62  63  65
  66  67  68  69  72  73  76  78  79  80  81  82  83  85  86  87  88  89
  90  91  94  95  98  99 100 101 102 103 104 105 106 107 109 111 113 114
 115 116 117 118 119 120 121 122 123] TEST: [  4   6   8  11  16  19  25  27  35  48  57  64  70  71  74  75  77  84
  92  93  96  97 108 110 112]
Scores from each Iteration:  [0.96]
Train: [  1   2   3   4   6   7   8   9  10  11  12  14  15  16  17  18  19  20
  21  22  23  24  25  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  42  45  46  47  48  50  51  52  54  55  56  57  59  61  62  64  65
  66  67  68  70  71  73  74  75  76  77  78  79  80  81  82  83  84  87
  88  90  91  92  93  96  97  98  99 100 103 104 105 106 107 108 109 110
 111 112 113 114 115 1

In [25]:
# Shapes of training set
print("Dataset (images) shape: {shape}".format(shape=images.shape))

Dataset (images) shape: (124, 128, 128, 30)


#### Training Without K Fold Cross Validation
#### Training (80%), Testing (20%)

In [31]:
indices = np.arange(124)
x_train,x_test,y_train,y_test, indices_train,indices_test =train_test_split(train_dataset,y,indices,test_size=0.2, random_state=13)





In [32]:
svclassifier = SVC(kernel='linear')
a=svclassifier.fit(x_train, y_train)
y_pred = svclassifier.predict(x_test)

#### Prediction results 

In [36]:
for i in range(25):
    print(indices_test[i] )
    print(data.filename[indices_test[i]]+' prediction:'+y_pred[i])


14
3235_1_functional.nii prediction:PD
89
4093_functional.nii prediction:control
55
3900_functional.nii prediction:control
85
4085_2_functional.nii prediction:control
105
4104_7_functional.nii prediction:control
15
3235_2_functional.nii prediction:PD
50
3662_1_functional.nii prediction:control
53
3665_functional.nii prediction:PD
87
4085_4_functional.nii prediction:control
39
3460_3_functional.nii prediction:control
30
3445_functional.nii prediction:control
42
3468_2_functional.nii prediction:control
111
4105_4_functional.nii prediction:control
12
3230_functional.nii prediction:control
90
4094_functional.nii prediction:PD
23
3423_functional.nii prediction:PD
57
3901_2_functional.nii prediction:control
43
3468_3_functional.nii prediction:control
59
3901_4_functional.nii prediction:control
113
4106_functional.nii prediction:PD
109
4105_2_functional.nii prediction:control
106
4104_8_functional.nii prediction:control
95
4100_3_functional.nii prediction:control
110
4105_3_functional.nii pre

#### Printing results

In [37]:
scores1=[]
scores1.append(a.score(x_test,y_test))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(scores1)

[[ 5  4]
 [ 2 14]]
              precision    recall  f1-score   support

          PD       0.71      0.56      0.63         9
     control       0.78      0.88      0.82        16

    accuracy                           0.76        25
   macro avg       0.75      0.72      0.72        25
weighted avg       0.75      0.76      0.75        25

[0.76]
